In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.externals import joblib
from sklearn.pipeline import Pipeline

<h1><font color="blue">Text Analytics</font></h1>

In [2]:
train_file = "./imdb_train.csv"
train_data = pd.read_csv(train_file)

In [3]:
vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1, 2), sublinear_tf=True, max_features=40000)
features_train = vectorizer.fit_transform(train_data.text)
labels_train = train_data.polarity

In [4]:
model = SGDClassifier(penalty="l1", loss="log", max_iter=200)
model.fit(features_train, labels_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', max_iter=200, n_iter=None,
       n_jobs=1, penalty='l1', power_t=0.5, random_state=None,
       shuffle=True, tol=None, verbose=0, warm_start=False)

In [5]:
model.score(features_train, labels_train)

0.85908

El transformador de features y el modelo estimado se empaquetan en un objeto Pipeline:

In [6]:
pipe = Pipeline([('vectorizer', vectorizer), ('classifier', model)])

In [7]:
test_text = train_data.loc[5, ['text']].tolist()
test_text

['"It appears that many critics find the idea of a Woody Allen drama unpalatable." And for good reason: they are unbearably wooden and pretentious imitations of Bergman. And let\'s not kid ourselves: critics were mostly supportive of Allen\'s Bergman pretensions, Allen\'s whining accusations to the contrary notwithstanding. What I don\'t get is this: why was Allen generally applauded for his originality in imitating Bergman, but the contemporaneous Brian DePalma was excoriated for "ripping off" Hitchcock in his suspense/horror films? In Robin Wood\'s view, it\'s a strange form of cultural snobbery. I would have to agree with that.']

In [8]:
print(pipe.predict_proba(test_text))
print(pipe.predict(test_text))

[[0.70985706 0.29014294]]
[0]


### Serialization

In [9]:
joblib.dump(pipe, './sentiment_model.pkl')

['./sentiment_model.pkl']

**Uso:**

In [10]:
import numpy as np
from sklearn.externals import joblib

model2 = joblib.load('./sentiment_model.pkl')

In [11]:
def scorer(text, classifier):
    pred = classifier.predict(text)[0]
    ppred = np.max(classifier.predict_proba(text))
    return {'sentiment': int(pred), 'probability': float(ppred)}

In [12]:
test_text = train_data.loc[5, ['text']].tolist()
test_text

['"It appears that many critics find the idea of a Woody Allen drama unpalatable." And for good reason: they are unbearably wooden and pretentious imitations of Bergman. And let\'s not kid ourselves: critics were mostly supportive of Allen\'s Bergman pretensions, Allen\'s whining accusations to the contrary notwithstanding. What I don\'t get is this: why was Allen generally applauded for his originality in imitating Bergman, but the contemporaneous Brian DePalma was excoriated for "ripping off" Hitchcock in his suspense/horror films? In Robin Wood\'s view, it\'s a strange form of cultural snobbery. I would have to agree with that.']

In [13]:
result = scorer(text=test_text, classifier=model2)
result

{'sentiment': 0, 'probability': 0.709857063917545}

<h1><font color="blue">Classification</font></h1>

In [ ]:
file = "./iris_data.csv"

In [ ]:
iris_data = pd.read_csv(file)
iris_data.head()

In [ ]:
iris_data[iris_data['type'] == 'virginica'].head()

In [ ]:
iris_model = LogisticRegression()
iris_model.fit(iris_data[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']], iris_data.type)

In [ ]:
iris_model.score(iris_data[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']], iris_data.type)

## Serialization

In [ ]:
joblib.dump(iris_model, './iris_model.pkl')

**Uso:**

In [ ]:
iris_model = joblib.load('./iris_model.pkl')

In [ ]:
def scorer(data, model):
    sepal_length, sepal_width = float(data['sepal_length']), float(data['sepal_width'])
    petal_length, petal_width = float(data['petal_length']), float(data['petal_width'])
    # deben ir en el mismo orden en que se estimó el modelo
    data_p = [[sepal_length, sepal_width, petal_length, petal_width]]
    pred = model.predict(data_p)[0]
    ppred = round(np.max(model.predict_proba(data_p)), 4)
    return {'predicted': pred, 'probability': float(ppred), 'message': 'success'}

In [ ]:
test_data = {'sepal_length': 4.9, 'sepal_width': 3, 'petal_length': 1.4, 'petal_width': 0.2}
print(scorer(test_data, iris_model))